In [1]:
import warnings
warnings.filterwarnings('ignore')

import torch
import random
import numpy as np
import torch.backends.cudnn as cudnn

def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(seed)
    
SEED = 333
seed_everything(SEED)

In [2]:
from src.dataload import get_inshop_dataset
from sklearn.model_selection import train_test_split

DATASET_DIR = '../../../dataset/InShop/'
all_data, label_enc, cat_enc, subcat_enc, vari_enc = get_inshop_dataset(root_path=DATASET_DIR)

X_train, X_valid, y_train, y_valid, cat_tr, cat_val = train_test_split(all_data, 
                                                            label_enc, 
                                                            cat_enc,
                                                            test_size=0.15, 
                                                            random_state=SEED, 
                                                            stratify=cat_enc)

X_valid, X_test, y_valid, y_test, _, _ = train_test_split(X_valid, 
                                                            y_valid, 
                                                            cat_val,
                                                            test_size=0.5, 
                                                            random_state=SEED, 
                                                            stratify=cat_val)

In [3]:
from src.utils import MyDataset, TransformsCE

size = 224
batch_size = 32

train_dataset = MyDataset(img_list=X_train, 
                          label_list=y_train, 
                          phase='train', 
                          transform=TransformsCE(size))

val_dataset = MyDataset(img_list=X_valid, 
                          label_list=y_valid, 
                          phase='val', 
                          transform=TransformsCE(size))

test_dataset = MyDataset(img_list=X_test, 
                          label_list=y_test, 
                          phase='test', 
                          transform=TransformsCE(size))

image_datasets = {'train' : train_dataset, 'val' : val_dataset, 'test': test_dataset}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}


train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
                                                                                                                                             
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

dataloaders = {'train': train_dataloader, 'val': val_dataloader, 'test': test_dataloader}

-----
### HIE

In [4]:
SEED = 333
seed_everything(SEED)

torch.cuda.empty_cache()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

MODEL_PATH = '../../../pretrained/ResNet50-HIE-33+300-400.pth'
SAVE_PATH = '../../../weights'

RUN_NAME = 'ResNet50-HIE-33-400-linear'
WANDB_PRJ = 'DeepFashion-pre-linear'

CONFIG = {
    "dataset": "DeepFashion",
    "machine": "offline cluster",
    "model": "ResNet50-HIE",
    "learning_rate": 0.2,
    "batch_size": batch_size*2
}

print(f'Training on {device} - {WANDB_PRJ} / {RUN_NAME}')

Training on cuda:0 - DeepFashion-pre-linear / ResNet50-HIE-33-400-linear


In [5]:
import timm

import torch.nn as nn

from src.models import MLP
from src.optim_lars import LARS

num_classes = 0
model = timm.create_model('resnet50', pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load(MODEL_PATH))

mlp = MLP(num_classes=17) 

criterion = nn.CrossEntropyLoss()

optimizer = LARS(
    list(mlp.parameters()),
    lr=0.2,
    weight_decay=1e-6,
    exclude_from_weight_decay=['batch_normalization', 'bias'],
)

warmupscheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch : (epoch+1)/10.0, verbose = True)
mainscheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 500, eta_min=0.05, last_epoch=-1, verbose = True)

Adjusting learning rate of group 0 to 2.0000e-02.
Epoch 00000: adjusting learning rate of group 0 to 2.0000e-01.


In [6]:
import wandb

from src.train import run_linearprotocol, run_test


wandb.init(name=RUN_NAME, project=WANDB_PRJ, config=CONFIG, reinit=True)

run_linearprotocol(model, mlp, dataloaders, dataset_sizes,
                   optimizer, criterion, warmupscheduler, mainscheduler, device, RUN_NAME, SAVE_PATH)


torch.cuda.empty_cache()

model = timm.create_model('resnet50', pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load(MODEL_PATH))

mlp = MLP(num_classes=17)
mlp.load_state_dict(torch.load(f'{SAVE_PATH}/{RUN_NAME}-MLP-best_model.pth'))  

run_test(model, mlp, dataloaders, dataset_sizes, criterion, device)

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 33h002. Use `wandb login --relogin` to force relogin


Adjusting learning rate of group 0 to 4.0000e-02.
e: 1 | Train Loss: 149824.8254 | 3.3439 | Acc: 0.6641
e: 1 | Valid Loss: 9653.5415 | 2.4421 | Acc: 0.7293
Validation loss decreased (inf --> 9653.541527).
Adjusting learning rate of group 0 to 6.0000e-02.
e: 2 | Train Loss: 100262.6889 | 2.2378 | Acc: 0.7816
e: 2 | Valid Loss: 8987.4838 | 2.2736 | Acc: 0.7442
Validation loss decreased (9653.541527 --> 8987.483818).
Adjusting learning rate of group 0 to 8.0000e-02.
e: 3 | Train Loss: 91467.4355 | 2.0415 | Acc: 0.7872
e: 3 | Valid Loss: 8302.0381 | 2.1002 | Acc: 0.7554
Validation loss decreased (8987.483818 --> 8302.038136).
Adjusting learning rate of group 0 to 1.0000e-01.
e: 4 | Train Loss: 82194.6284 | 1.8345 | Acc: 0.7993
e: 4 | Valid Loss: 7661.6352 | 1.9382 | Acc: 0.7647
Validation loss decreased (8302.038136 --> 7661.635233).
Adjusting learning rate of group 0 to 1.2000e-01.
e: 5 | Train Loss: 73196.1758 | 1.6337 | Acc: 0.8141
e: 5 | Valid Loss: 7107.0442 | 1.7979 | Acc: 0.7774
Val

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_acc,▁▅▆▇▇▇▇▇▇███████████████████████████████
epoch_loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
running_loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_loss,▁
test_running_loss,▁
val_acc,▁▃▅▆▇▇▇█████████████████████████████████
val_loss,█▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_running_loss,█▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,101


-------
### SupCL

In [7]:
SEED = 333
seed_everything(SEED)

torch.cuda.empty_cache()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

MODEL_PATH = '../../../pretrained/ResNet50-SupCL-33+300-400.pth'
SAVE_PATH = '../../../weights'

RUN_NAME = 'ResNet50-SupCL-33-400-linear'
WANDB_PRJ = 'DeepFashion-pre-linear'

CONFIG = {
    "dataset": "DeepFashion",
    "machine": "offline cluster",
    "model": "ResNet50-SupCL",
    "learning_rate": 0.2,
    "batch_size": batch_size*2
}

print(f'Training on {device} - {WANDB_PRJ} / {RUN_NAME}')

Training on cuda:0 - DeepFashion-pre-linear / ResNet50-SupCL-33-400-linear


In [8]:
import timm

import torch.nn as nn

from src.models import MLP
from src.optim_lars import LARS

num_classes = 0
model = timm.create_model('resnet50', pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load(MODEL_PATH))

mlp = MLP(num_classes=17) 

criterion = nn.CrossEntropyLoss()

optimizer = LARS(
    list(mlp.parameters()),
    lr=0.2,
    weight_decay=1e-6,
    exclude_from_weight_decay=['batch_normalization', 'bias'],
)

warmupscheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch : (epoch+1)/10.0, verbose = True)
mainscheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 500, eta_min=0.05, last_epoch=-1, verbose = True)

Adjusting learning rate of group 0 to 2.0000e-02.
Epoch 00000: adjusting learning rate of group 0 to 2.0000e-01.


In [9]:
import wandb

from src.train import run_linearprotocol, run_test


wandb.init(name=RUN_NAME, project=WANDB_PRJ, config=CONFIG, reinit=True)

run_linearprotocol(model, mlp, dataloaders, dataset_sizes,
                   optimizer, criterion, warmupscheduler, mainscheduler, device, RUN_NAME, SAVE_PATH)


torch.cuda.empty_cache()

model = timm.create_model('resnet50', pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load(MODEL_PATH))

mlp = MLP(num_classes=17)
mlp.load_state_dict(torch.load(f'{SAVE_PATH}/{RUN_NAME}-MLP-best_model.pth'))  

run_test(model, mlp, dataloaders, dataset_sizes, criterion, device)

wandb.finish()

Adjusting learning rate of group 0 to 4.0000e-02.
e: 1 | Train Loss: 89719.7514 | 2.0024 | Acc: 0.8130
e: 1 | Valid Loss: 6538.3100 | 1.6540 | Acc: 0.8174
Validation loss decreased (inf --> 6538.309963).
Adjusting learning rate of group 0 to 6.0000e-02.
e: 2 | Train Loss: 47838.4621 | 1.0677 | Acc: 0.8668
e: 2 | Valid Loss: 6519.8922 | 1.6494 | Acc: 0.8158
Validation loss decreased (6538.309963 --> 6519.892226).
Adjusting learning rate of group 0 to 8.0000e-02.
e: 3 | Train Loss: 46777.3448 | 1.0440 | Acc: 0.8657
e: 3 | Valid Loss: 6503.4688 | 1.6452 | Acc: 0.8158
Validation loss decreased (6519.892226 --> 6503.468764).
Adjusting learning rate of group 0 to 1.0000e-01.
e: 4 | Train Loss: 45517.6220 | 1.0159 | Acc: 0.8662
e: 4 | Valid Loss: 6511.9045 | 1.6473 | Acc: 0.8168
Adjusting learning rate of group 0 to 1.2000e-01.
e: 5 | Train Loss: 43960.4647 | 0.9812 | Acc: 0.8701
e: 5 | Valid Loss: 6515.3044 | 1.6482 | Acc: 0.8179
Adjusting learning rate of group 0 to 1.4000e-01.
e: 6 | Train

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_acc,▁▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇██████████████████
epoch_loss,█▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
running_loss,█▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_loss,▁
test_running_loss,▁
val_acc,▇▄▆▄▂▄▃▃▄▅▆▅▃▄▁▂▄▃▂▅▇▄▄▆▇▄▆▄▃▆▅▂▅▅▆▇█▅▆█
val_loss,██▇▇▇▇▆▅▆▄▅▄▅▃▅▄▃▄▄▄▄▄▃▃▄▄▄▃▃▅▄▁▃▄▄▄▃▄▄▂
val_running_loss,██▇▇▇▇▆▅▆▄▅▄▅▃▅▄▃▄▄▄▄▄▃▃▄▄▄▃▃▅▄▁▃▄▄▄▃▄▄▂
epoch,101
